In [11]:
import sys

sys.path.append("../../src/stong_core_cython")

In [12]:
import numpy as np
import pandas as pd
import hartree_fock
import basis_data
import importlib
from scipy.optimize import minimize

importlib.reload(hartree_fock)
importlib.reload(basis_data)

from atom import *
from hartree_fock import *
from basis_data import *

In [13]:
df = pd.read_csv("../make_base/sto_3g.csv")

In [14]:
def solve(zetas, atom_symbol):
    atoms = [Atom(atom_symbol, np.array([0.0, 0.0, 0.0]))]
    molecule = Molecule(atoms)
    basis_data = {atom_symbol: create_basis_data(df, zetas, 2, 1)}
    hartreefock = HartreeFock(molecule, basis_data, max_iter=10000, tol=1e-10)
    hartreefock.scf()
    hartreefock.calculate_total_energy()
    results = hartreefock.get_results()
    return results, hartreefock


def get_energy(zetas, atom_symbol):
    results, _ = solve(zetas, atom_symbol)
    return results["energy"]

In [15]:
elements = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne"]
zetas = {}
for element in elements:
    print(element)
    results = minimize(get_energy, [2.69, 0.75], args=(element))
    zetas[element] = results.x
    print(zetas[element])

H
[1.34826568 1.18246847]
He
[2.41160998 1.77710959]
Li
[2.69209868 0.65224709]
Be
[3.6918977  0.96952017]
B
[4.69250686 1.28008946]
C
[5.69264837 1.58789393]
N
[6.69122752 1.91394711]
O
[7.68983164 2.23314354]
F
[8.68731557 2.56256126]
Ne
[9.68505199 2.88587467]


In [16]:
df_zetas = pd.DataFrame(zetas)
df_zetas.to_csv("zetas.csv", index=False)